In [1]:
import pandas as pd
import requests
import csv
import io, os
import datetime
import sqlalchemy
import psycopg2
from sqlalchemy import create_engine

In [2]:
def mostrar_unicos(df):
    return pd.DataFrame({'Cantidad de valores unicos':df.agg(lambda x: len(pd.unique(x))), 
              'Valores unicos':df.agg(lambda x: (pd.unique(x)))
             })

def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b)
    return s

def columns_to_lower(df):
    lower_name = []
    for name in df.columns:
        lower_name.append(normalize(name.lower()))
    return lower_name

In [3]:
url_bibliotecas = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/01c6c048-dbeb-44e0-8efa-6944f73715d7/download/biblioteca_popular.csv'
url_cines = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/392ce1a8-ef11-4776-b280-6f1c7fae16ae/download/cine.csv'
url_museos = 'https://datos.cultura.gob.ar/dataset/37305de4-3cce-4d4b-9d9a-fec3ca61d09f/resource/4207def0-2ff7-41d5-9095-d42ae8207a5d/download/museos_datosabiertos.csv'

response_bibliotecas = requests.get(url_bibliotecas).content
response_cines = requests.get(url_cines).content
response_museos = requests.get(url_museos).content

bibliotecas = pd.read_csv(io.StringIO(response_bibliotecas.decode('utf-8')))
cines = pd.read_csv(io.StringIO(response_cines.decode('utf-8')))
museos = pd.read_csv(io.StringIO(response_museos.decode('utf-8')))

#### Organizar los archivos en rutas siguiendo la siguiente estructura: “categoría\año-mes\categoria-dia-mes-año.csv”
 - Por ejemplo: “museos\2021-noviembre\museos-03-11-2021”
 - Si el archivo existe debe reemplazarse. La fecha de la nomenclatura es la fecha de descarga.


In [4]:
categorias = {'cines': cines , 'museos' : museos, 'bibliotecas' : bibliotecas}
today = datetime.date.today()

In [5]:
cines.head(2)

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,Categoría,Provincia,Departamento,Localidad,Nombre,Dirección,...,Información adicional,Latitud,Longitud,TipoLatitudLongitud,Fuente,tipo_gestion,Pantallas,Butacas,espacio_INCAA,año_actualizacion
0,10049030,10,10049,NaN,Salas de cine,Catamarca,Capital,Catamarca,Cinemacenter,Intendente Mamerto Medina 220,...,NaN,-28.464737,-65.800675,Localización precisa,INCAA / SInCA,Privado comercial,5,743,NaN,2018
1,10091150,10,10091,NaN,Salas de cine,Catamarca,Santa Maria,Santa María,Centro Cultural San Agustín,San Martin 173,...,NaN,-26.693859,-66.049058,Localización precisa,INCAA / SInCA,Privado comercial,1,440,0,2018


In [6]:
bibliotecas.head(2)

,Cod_Loc,IdProvincia,IdDepartamento,Observacion,Categoría,Subcategoria,Provincia,Departamento,Localidad,Nombre,...,Mail,Web,Información adicional,Latitud,Longitud,TipoLatitudLongitud,Fuente,Tipo_gestion,año_inicio,Año_actualizacion
0,70049060,70,70049,NaN,Bibliotecas Populares,NaN,San Juan,Iglesia,Rodeo,Biblioteca Popular Juan P. Garramuno,...,s/d,NaN,NaN,-30.209250,-69.130117,Localización precisa,CONABIP,Comunitaria,1930.0,2018
1,2000010,2,2000,NaN,Bibliotecas Populares,NaN,Ciudad Autónoma de Buenos Aires,Ciudad Autonoma de Buenos Aires,Ciudad de Buenos Aires,Biblioteca Popular Helena Larroque de Roffo,...,asociacionroffo@yahoo.com.ar,NaN,NaN,-34.598461,-58.494690,Localización precisa,CONABIP,Comunitaria,1916.0,2018


In [7]:
museos.head(2)

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,...,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,año_inauguracion,actualizacion
0,6588100,6,6588,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,9 de Julio,Archivo Y Museo Histórico Gral. Julio De Vedia,Libertad 1191,...,archivoymuseo@yahoo.com.ar,www.portaldel9.com.ar,-35.441762,-60.887598,Localización precisa,NaN,DNPyM,Municipal,1920.0,2017
1,6077010,6,6077,NaN,Espacios de Exhibición Patrimonial,Museos,Buenos Aires,Arrecifes,Museo Y Archivo Histórico De Arrecifes,Gerardo Risso y España,...,museoarrecifes@hotmail.com,NaN,-34.066970,-60.102555,Localización precisa,NaN,DNPyM,Municipal,1972.0,2017


cod_localidad
id_provincia
id_departamento
categoría
provincia
localidad
nombre
domicilio
código_postal
número_de_teléfono 
mail
web

In [8]:
bibliotecas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Cod_Loc                2017 non-null   int64  
 1   IdProvincia            2017 non-null   int64  
 2   IdDepartamento         2017 non-null   int64  
 3   Observacion            0 non-null      float64
 4   Categoría              2017 non-null   object 
 5   Subcategoria           0 non-null      float64
 6   Provincia              2017 non-null   object 
 7   Departamento           2017 non-null   object 
 8   Localidad              2017 non-null   object 
 9   Nombre                 2017 non-null   object 
 10  Domicilio              2017 non-null   object 
 11  Piso                   0 non-null      float64
 12  CP                     2017 non-null   object 
 13  Cod_tel                2017 non-null   object 
 14  Teléfono               2017 non-null   object 
 15  Mail

In [9]:
museos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1182 entries, 0 to 1181
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cod_Loc              1182 non-null   int64  
 1   IdProvincia          1182 non-null   int64  
 2   IdDepartamento       1182 non-null   int64  
 3   Observaciones        0 non-null      float64
 4   categoria            1182 non-null   object 
 5   subcategoria         1049 non-null   object 
 6   provincia            1182 non-null   object 
 7   localidad            1182 non-null   object 
 8   nombre               1182 non-null   object 
 9   direccion            1164 non-null   object 
 10  piso                 0 non-null      float64
 11  CP                   1116 non-null   object 
 12  cod_area             989 non-null    float64
 13  telefono             988 non-null    object 
 14  Mail                 264 non-null    object 
 15  Web                  75 non-null     o

In [10]:
cines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Cod_Loc                329 non-null    int64  
 1   IdProvincia            329 non-null    int64  
 2   IdDepartamento         329 non-null    int64  
 3   Observaciones          0 non-null      float64
 4   Categoría              329 non-null    object 
 5   Provincia              329 non-null    object 
 6   Departamento           329 non-null    object 
 7   Localidad              329 non-null    object 
 8   Nombre                 329 non-null    object 
 9   Dirección              329 non-null    object 
 10  Piso                   329 non-null    object 
 11  CP                     329 non-null    int64  
 12  cod_area               329 non-null    object 
 13  Teléfono               329 non-null    object 
 14  Mail                   329 non-null    object 
 15  Web   

In [11]:
cines.columns = columns_to_lower(cines)
museos.columns = columns_to_lower(museos)
bibliotecas.columns = columns_to_lower(bibliotecas)

In [12]:
mostrar_unicos(cines)

,Cantidad de valores unicos,Valores unicos
cod_loc,180,"[10049030, 10091150, 22140060, 26091020, 18007..."
idprovincia,24,"[10, 22, 26, 18, 30, 38, 42, 46, 54, 66, 70, 7..."
iddepartamento,153,"[10049, 10091, 22140, 26091, 18007, 30056, 300..."
observaciones,1,[nan]
categoria,1,[Salas de cine]
provincia,24,"[Catamarca, Chaco, Chubut, Corrientes, Entre R..."
departamento,134,"[Capital, Santa Maria, San Fernando, Sarmiento..."
localidad,190,"[Catamarca, Santa María, Resistencia, Sarmient..."
nombre,243,"[Cinemacenter, Centro Cultural San Agustín, Lo..."
direccion,329,"[Intendente Mamerto Medina 220, San Martin 173..."


In [13]:
mostrar_unicos(museos)

,Cantidad de valores unicos,Valores unicos
cod_loc,479,"[6588100, 6077010, 6035010, 6049020, 6056010, ..."
idprovincia,24,"[6, 10, 22, 26, 2, 14, 18, 30, 34, 38, 42, 46,..."
iddepartamento,284,"[6588, 6077, 6035, 6049, 6056, 6063, 6490, 607..."
observaciones,1,[nan]
categoria,1,[Espacios de Exhibición Patrimonial]
subcategoria,2,"[Museos, nan]"
provincia,24,"[Buenos Aires, Catamarca, Chaco, Chubut, Ciuda..."
localidad,518,"[9 de Julio, Arrecifes, Avellaneda, Azul, Bahí..."
nombre,1133,[Archivo Y Museo Histórico Gral. Julio De Vedi...
direccion,1120,"[Libertad 1191, Gerardo Risso y España, Calle ..."


In [14]:
mostrar_unicos(bibliotecas)

,Cantidad de valores unicos,Valores unicos
cod_loc,1083,"[70049060, 2000010, 6049020, 6462040, 6007010,..."
idprovincia,24,"[70, 2, 6, 30, 82, 18, 14, 74, 86, 90, 50, 46,..."
iddepartamento,442,"[70049, 2000, 6049, 6462, 6007, 6028, 6105, 66..."
observacion,1,[nan]
categoria,1,[Bibliotecas Populares]
subcategoria,1,[nan]
provincia,25,"[San Juan, Ciudad Autónoma de Buenos Aires, Bu..."
departamento,377,"[Iglesia, Ciudad Autonoma de Buenos Aires, Azu..."
localidad,1196,"[Rodeo, Ciudad de Buenos Aires, Azul, Juan Bau..."
nombre,1452,"[Biblioteca Popular Juan P. Garramuno, Bibliot..."


In [15]:
for elemento in museos.columns:
    if(elemento in cines.columns and elemento in bibliotecas.columns):
        print(elemento,' OK')
    else:
        print(elemento,' --')

cod_loc  OK
idprovincia  OK
iddepartamento  OK
observaciones  --
categoria  OK
subcategoria  --
provincia  OK
localidad  OK
nombre  OK
direccion  --
piso  OK
cp  OK
cod_area  --
telefono  OK
mail  OK
web  OK
latitud  OK
longitud  OK
tipolatitudlongitud  OK
info_adicional  --
fuente  OK
jurisdiccion  --
año_inauguracion  --
actualizacion  --


In [16]:
bibliotecas.rename(columns = {'domicilio':'direccion'}, inplace=True)

In [17]:
for elemento in museos.columns:
    if(elemento in cines.columns and elemento in bibliotecas.columns):
        print(elemento,' OK')
    else:
        print(elemento,' --')

cod_loc  OK
idprovincia  OK
iddepartamento  OK
observaciones  --
categoria  OK
subcategoria  --
provincia  OK
localidad  OK
nombre  OK
direccion  OK
piso  OK
cp  OK
cod_area  --
telefono  OK
mail  OK
web  OK
latitud  OK
longitud  OK
tipolatitudlongitud  OK
info_adicional  --
fuente  OK
jurisdiccion  --
año_inauguracion  --
actualizacion  --


In [18]:
need_columns = ['cod_loc','idprovincia','iddepartamento','categoria','provincia','localidad','nombre','direccion','cp','telefono','mail','web','fuente']

In [19]:
data = pd.concat([museos[need_columns],cines[need_columns],bibliotecas[need_columns]], ignore_index = True)

In [20]:
print([museos[need_columns].shape,cines[need_columns].shape,bibliotecas[need_columns].shape])
print([museos[need_columns].shape[0]+ cines[need_columns].shape[0]+ bibliotecas[need_columns].shape[0]])

[(1182, 13), (329, 13), (2017, 13)]
[3528]


In [21]:
data.shape

(3528, 13)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3528 entries, 0 to 3527
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cod_loc         3528 non-null   int64 
 1   idprovincia     3528 non-null   int64 
 2   iddepartamento  3528 non-null   int64 
 3   categoria       3528 non-null   object
 4   provincia       3528 non-null   object
 5   localidad       3528 non-null   object
 6   nombre          3528 non-null   object
 7   direccion       3510 non-null   object
 8   cp              3462 non-null   object
 9   telefono        3334 non-null   object
 10  mail            2610 non-null   object
 11  web             404 non-null    object
 12  fuente          3528 non-null   object
dtypes: int64(3), object(10)
memory usage: 358.4+ KB


In [23]:
data.groupby(['provincia'])['idprovincia'].value_counts()

provincia                                              idprovincia
Buenos Aires                                           6              801
Catamarca                                              10              74
Chaco                                                  22             107
Chubut                                                 26             102
Ciudad Autónoma de Buenos Aires                        2              210
Corrientes                                             18              80
Córdoba                                                14             346
Entre Ríos                                             30             128
Formosa                                                34              34
Jujuy                                                  38              91
La Pampa                                               42             123
La Rioja                                               46              46
Mendoza                                      

In [26]:
salta_mask = data.provincia == 'Salta'
data.loc[salta_mask,'idprovincia'] = data[salta_mask]['idprovincia'].map({58:66,66:66})

sfe_mask = data.provincia == 'Santa Fé'
data.loc[sfe_mask,'provincia'] = data[sfe_mask]['provincia'].map({'Santa Fé':'Santa Fe'})

data.loc[data.idprovincia == 58,'provincia'] = 'Neuquén'
data.loc[data.idprovincia == 94,'provincia'] ='Tierra del Fuego, Antártida e Islas del Atlántico Sur'

In [27]:
data.groupby(['provincia'])['idprovincia'].value_counts()

provincia                                              idprovincia
Buenos Aires                                           6              801
Catamarca                                              10              74
Chaco                                                  22             107
Chubut                                                 26             102
Ciudad Autónoma de Buenos Aires                        2              210
Corrientes                                             18              80
Córdoba                                                14             346
Entre Ríos                                             30             128
Formosa                                                34              34
Jujuy                                                  38              91
La Pampa                                               42             123
La Rioja                                               46              46
Mendoza                                      

#### Procesar los datos conjuntos para poder generar una tabla con la siguiente información:

In [44]:
data = data.rename(columns = {
    'cod_loc' : 'cod_localidad',
    'idprovincia' : 'id_provincia',
    'iddepartamento' : 'id_departamento',
    'direccion' : 'domicilio' 
    })

In [54]:
data.groupby(['categoria','fuente','provincia']).count().head(50)

cod_localidad  \
categoria                          fuente                           provincia                                                           
Bibliotecas Populares              CONABIP                          Buenos Aires                                                  543   
                                                                    Catamarca                                                      38   
                                                                    Chaco                                                          70   
                                                                    Chubut                                                         48   
                                                                    Ciudad Autónoma de Buenos Aires                                43   
                                                                    Corrientes                                                     58   
                                                                    Córdoba                                                       185   
                                                                    Entre Ríos                                                     58   
                                                                    Formosa                                                        25   
                                                                    Jujuy                                                          40   
                                                                    La Pampa                                                       72   
                                                                    La Rioja                                                       24   
                                                                    Mendoza                                                        67   
                                                                    Misiones                                                       40   
                                                                    Neuquén                                                        70   
                                                                    Río Negro                                                      60   
                                                                    Salta                                                          64   
                                                                    San Juan                                                       53   
                                                                    San Luis                                                       45   
                                                                    Santa Cruz                                                     21   
                                                                    Santa Fe                                                      273   
                                                                    Santiago del Estero                                            67   
                                                                    Tierra del Fuego, Antártida e Islas del Atlánti...              9   
                                                                    Tucumán                                                        37   
                                   Gob. Pcia.                       San Juan                                                        1   
                                   Secretaria de Cultura Provincial Río Negro                                                       6   
Espacios de Exhibición Patrimonial CNMLH - Enlace SInCA             Catamarca                                                       1   
                                   CNMLH - Ente Cultural de Tucumán Tucumán                                                         1   
          

- Cantidad de registros totales por categoría

In [46]:
data.groupby('categoria')['categoria'].count().to_frame().rename(columns = {'categoria':'cantidad'}).reset_index()

,categoria,cantidad
0,Bibliotecas Populares,2017
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,329


- Cantidad de registros totales por fuente


In [47]:
data.groupby('fuente')['fuente'].count().to_frame().rename(columns = {'fuente':'cantidad'}).reset_index()

,fuente,cantidad
0,CNMLH - Enlace SInCA,1
1,CNMLH - Ente Cultural de Tucumán,1
2,CONABIP,2010
3,DNPyM,1048
4,Dirección Provincial de Patrimonio Cultural - ...,1
5,Dirección de Cultura de Villa Allende,2
6,Dirección de Cultura y de Educación de la Muni...,1
7,Gob. Pcia.,1
8,Gobierno de la Provincia,14
9,Gobierno de la Provincia de Chubut / SInCA 2013,3


- Cantidad de registros por provincia y categoría

In [48]:
data.groupby(['provincia','categoria'])['categoria'].count().to_frame().rename(columns = {'categoria':'cantidad'}).reset_index()

,provincia,categoria,cantidad
0,Buenos Aires,Bibliotecas Populares,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,106
3,Catamarca,Bibliotecas Populares,38
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
67,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
68,"Tierra del Fuego, Antártida e Islas del Atlánt...",Salas de cine,3
69,Tucumán,Bibliotecas Populares,37
70,Tucumán,Espacios de Exhibición Patrimonial,26


##### Procesar la información de cines para poder crear una tabla que contenga: 
- Provincia
- Cantidad de pantallas
- Cantidad de butacas
- Cantidad de espacios INCAA

In [49]:
cines.groupby('provincia')[['pantallas','butacas','espacio_incaa']].count().reset_index()

,provincia,pantallas,butacas,espacio_incaa
0,Buenos Aires,106,106,20
1,Catamarca,4,4,2
2,Chaco,4,4,1
3,Chubut,9,9,4
4,Ciudad Autónoma de Buenos Aires,35,35,3
5,Corrientes,8,8,1
6,Córdoba,35,35,2
7,Entre Ríos,13,13,2
8,Formosa,2,2,1
9,Jujuy,4,4,2


In [50]:
data.describe()

,cod_localidad,id_provincia,id_departamento
count,3.528000e+03,3528.000000,3528.000000
mean,3.780012e+07,37.653061,37800.047336
std,3.037392e+07,30.468562,30373.901491
min,2.000010e+06,2.000000,2000.000000
25%,6.776030e+06,6.000000,6776.000000
50%,3.004904e+07,30.000000,30049.000000
75%,6.605622e+07,66.000000,66056.000000
max,9.401402e+07,94.000000,94014.000000
